In [1]:
import numpy as np
import pandas as pd
import matplotlib
import requests

In [2]:
stocks = ['aapl','amzn','fb']
ebitda=[]
gross_profit=[]
revenue=[]
enterprise_value=[]
debt=[]
for stock in stocks:
    url = f'http://iex.lewagon.com/stable/stock/{stock}/financials?period=quarter'
    response = requests.get(url).json()
    ebitda.append(response['financials'][0]['EBITDA']/1000000)
    gross_profit.append(response['financials'][0]['grossProfit']/1000000)
    revenue.append(response['financials'][0]['revenue']/1000000)
    debt.append(response['financials'][0]['totalDebt']/1000000)

In [3]:
market_cap=[]
for stock in stocks:
    url = f'http://iex.lewagon.com/stable/stock/{stock}/stats/'
    response = requests.get(url).json()
    market_cap.append(response['marketcap']/1000000)

In [4]:
df=pd.DataFrame(index=stocks)
df['Revenue'] = revenue
df['Gross Profit'] = gross_profit
df['EBITDA'] = ebitda
df['EBITDA margin'] = df['EBITDA']/df['Revenue']
df['Total debt'] = debt
df['Market Cap']= market_cap
df

,Revenue,Gross Profit,EBITDA,EBITDA margin,Total debt,Market Cap
aapl,111439.0,44328.0,36200.0,0.324841,112043.0,2.145351e+06
amzn,125555.0,46271.0,25549.0,0.203489,31816.0,1.584285e+06
fb,28071.0,22861.0,14638.0,0.521463,0.0,7.560532e+05
